In [3]:

import pandas as pd
import numpy as np


#upload this file /Users/tommi/Desktop/Uni_documents/Data/Proxies_Dragon.csv
proxies = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/Proxies_Dragon.csv')

#upload this file /Users/tommi/Desktop/Uni_documents/Data/Benchmarks_Dragon.csv
benchmarks = pd.read_csv('/Users/tommi/Desktop/Uni_documents/Data/Benchmarks_Dragon.csv')



In [2]:
proxies.head()  


,Dates,SPY US Equity,ACWI US Equity,IVV US Equity,TLT US Equity,TLH US Equity,ZROZ US Equity,GLD US Equity,IAU US Equity,PHYS US Equity,DBC US Equity,PDBC US Equity,GSG US Equity,BCI US Equity,VIXM US Equity,VXX US Equity,VXZ US Equity,UVXY US Equity
0,02/01/2015,205.43,58.36,206.73,127.32,136.26,121.49,114.08,23.00,9.82,18.23,21.1600,21.22,NaN,63.37,NaN,NaN,30412500.0
1,05/01/2015,201.72,57.21,203.10,129.32,137.47,124.72,115.80,23.36,9.97,17.97,20.9000,20.62,NaN,66.60,NaN,NaN,34787500.0
2,06/01/2015,199.82,56.62,201.27,131.65,138.74,127.98,117.12,23.58,10.07,17.80,20.6471,20.28,NaN,66.90,NaN,NaN,36337500.0
3,07/01/2015,202.31,57.36,203.76,131.39,138.60,128.21,116.43,23.46,10.02,17.69,20.5257,20.22,NaN,65.25,NaN,NaN,34037500.0
4,08/01/2015,205.90,58.26,207.40,129.65,137.59,125.09,115.94,23.38,9.97,17.76,20.5400,20.29,NaN,62.56,NaN,NaN,29850000.0


In [4]:
benchmarks.head()

,Dates,URTH US Equity,IEF US Equity,XAU Curncy,BCOM Index,VIX Index
0,02/01/2015,71.650,106.53,1188.39,103.8614,17.79
1,05/01/2015,69.970,107.18,1204.86,103.8730,19.92
2,06/01/2015,69.040,107.90,1218.58,104.0825,21.12
3,07/01/2015,69.740,107.88,1211.41,103.5252,19.31
4,08/01/2015,71.288,107.44,1208.79,103.6365,17.01


In [6]:
import pandas as pd
import statsmodels.api as sm


# Merge datasets on Dates to align time series
df = pd.merge(proxies, benchmarks, on="Dates", how="inner")

# Define asset classes and corresponding securities
asset_classes = {
    "Equities": {
        "benchmark": "URTH US Equity",
        "proxies": ["SPY US Equity", "ACWI US Equity", "IVV US Equity"]
    },
    "Bonds": {
        "benchmark": "IEF US Equity",
        "proxies": ["TLT US Equity", "TLH US Equity", "ZROZ US Equity"]
    },
    "Gold": {
        "benchmark": "XAU Curncy",
        "proxies": ["GLD US Equity", "IAU US Equity", "PHYS US Equity"]
    },
    "Commodities": {
        "benchmark": "BCOM Index",
        "proxies": ["DBC US Equity", "PDBC US Equity", "GSG US Equity", "BCI US Equity"]
    },
    "Volatility": {
        "benchmark": "VIX Index",
        "proxies": ["VIXM US Equity", "VXX US Equity", "VXZ US Equity", "UVXY US Equity"]
    }
}

# Function to run regressions and evaluate proxies
def evaluate_proxies(df, asset_classes):
    results = {}

    for category, assets in asset_classes.items():
        benchmark = assets["benchmark"]
        proxies = assets["proxies"]
        
        print(f"\n### {category} Regression Analysis ###")
        
        benchmark_returns = df[benchmark].pct_change().dropna()  # Compute returns
        best_r2 = -1
        best_proxy = None
        
        for proxy in proxies:
            proxy_returns = df[proxy].pct_change().dropna()
            aligned_df = pd.concat([benchmark_returns, proxy_returns], axis=1).dropna()
            aligned_df.columns = ["Benchmark", "Proxy"]

            X = sm.add_constant(aligned_df["Proxy"])  # Add constant for regression
            y = aligned_df["Benchmark"]
            model = sm.OLS(y, X).fit()
            
            r_squared = model.rsquared
            beta = model.params["Proxy"]
            tracking_error = (y - model.predict(X)).std()

            print(f"{proxy} - R²: {r_squared:.4f}, Beta: {beta:.4f}, Tracking Error: {tracking_error:.4f}")

            # Select the best proxy based on R²
            if r_squared > best_r2:
                best_r2 = r_squared
                best_proxy = proxy
        
        results[category] = best_proxy
        print(f"✅ Best Proxy for {category}: {best_proxy} (R²={best_r2:.4f})\n")
    
    return results

# Run the analysis
best_proxies = evaluate_proxies(df, asset_classes)



### Equities Regression Analysis ###
SPY US Equity - R²: 0.9328, Beta: 0.9476, Tracking Error: 0.0028
ACWI US Equity - R²: 0.9674, Beta: 0.9982, Tracking Error: 0.0019
IVV US Equity - R²: 0.9363, Beta: 0.9393, Tracking Error: 0.0027
✅ Best Proxy for Equities: ACWI US Equity (R²=0.9674)


### Bonds Regression Analysis ###
TLT US Equity - R²: 0.8415, Beta: 0.4046, Tracking Error: 0.0017
TLH US Equity - R²: 0.8886, Beta: 0.5745, Tracking Error: 0.0014
ZROZ US Equity - R²: 0.7722, Beta: 0.2616, Tracking Error: 0.0020
✅ Best Proxy for Bonds: TLH US Equity (R²=0.8886)


### Gold Regression Analysis ###
GLD US Equity - R²: 0.9631, Beta: 0.9739, Tracking Error: 0.0017
IAU US Equity - R²: 0.9596, Beta: 0.9722, Tracking Error: 0.0017
PHYS US Equity - R²: 0.8880, Beta: 0.9118, Tracking Error: 0.0029
✅ Best Proxy for Gold: GLD US Equity (R²=0.9631)


### Commodities Regression Analysis ###
DBC US Equity - R²: 0.7672, Beta: 0.7134, Tracking Error: 0.0044
PDBC US Equity - R²: 0.5331, Beta: 0.5154, 